In [1]:
# import modules
import warnings
warnings.filterwarnings('ignore')

from pymongo import MongoClient as MC
import time
import pandas as pd
from collections import Counter

In [2]:
# Create connection to Mlab
mongoURI = open('../../config/mongo_uri.txt').readlines()[0]
client = MC(mongoURI, connectTimeoutMS=30000)

FileNotFoundError: [Errno 2] No such file or directory: '../../config/mongo_uri.txt'

In [ ]:
# Get keys
keys = [i.rstrip() for i in open('../../config/keys.txt')]

In [ ]:
def get_db(client):
    return client['cpublica_sondeo']

def get_collection(db):
    return db.responses

def get_data(client):
    db = get_db(client)
    collection = get_collection(db)
    return [obj for obj in collection.find()]

In [ ]:
def create_dataset(client):
    exceptions = ['__v', '_id']
    data = get_data(client)
    for obj in data:
        for i in exceptions:
            del obj[i]
    
    return data

In [ ]:
def create_dataframe(data, path, k=keys):
    df_ = pd.DataFrame(data)
    df = df_[keys]
    df.to_excel(path.format('respuestas_' + str(int(time.time()))), index=False)
    
    return df

In [ ]:
# Create frequencies
def count_items(data, column, path):
    '''
    '''
    df_ = data.copy()
    df_[column] = df_[column].str.upper()
    
    count = df_[column].tolist()
    freqs = Counter(count).most_common()
    
    # Create dataframe
    cols = [column, 'frecuencia']
    df = pd.DataFrame(freqs, columns=cols)
    
    # Create new columns
    df['freqs_cumsum'] = df.frecuencia.cumsum()
    df['prcnt_cumsum'] = round((df.freqs_cumsum / df.frecuencia.sum()) * 100, 2)
    df['prcnt'] = round((df.frecuencia / df.frecuencia.sum()) * 100, 2)
    
    dataset = df.sort_values(by='frecuencia', ascending=False)
    dataset.to_excel(path.format('respuestas_counter_' + str(int(time.time()))), index=False)
    
    return dataset

In [ ]:
path = '../../usuarios_sondeo/responses_sondeo/{}.xlsx'

In [ ]:
data = create_dataset(client)
df = create_dataframe(data, path)
counter = count_items(df, 'name', path)